In [5]:
import twse_stock_day_crawler as tc
import crawlers
import pandas as pd

In [8]:
list_0050=crawlers.get_0050_list()

In [14]:

stock_list=list_0050[:2]
year_list=list(range(2021,2023))
month_list=list(range(1,3))

total_df=pd.DataFrame()
empty_df=pd.DataFrame()
for stock_id in stock_list:
    for y in year_list:
        for m in month_list:
            print(stock_id,y,m)
            
            df=tc.get_df(stock_id,y,m,empty_df)
            total_df=pd.concat([total_df,df],axis=0,ignore_index=True)
                    

2330 2021 1
2330 2021 2
2330 2022 1
2330 2022 2
2454 2021 1
2454 2021 2
2454 2022 1
2454 2022 2


In [15]:
total_df

,date,volume,value,open,high,low,close,record,stock_id
0,2021-01-04,39489959,21127581248,530.0,540.0,528.0,536.0,33316,2330
1,2021-01-05,34839391,18761831567,536.0,542.0,535.0,542.0,28512,2330
2,2021-01-06,55614434,30572783229,555.0,555.0,541.0,549.0,55462,2330
3,2021-01-07,53392763,30018630685,554.0,570.0,553.0,565.0,47905,2330
4,2021-01-08,62957148,36339702855,580.0,580.0,571.0,580.0,56426,2330
...,...,...,...,...,...,...,...,...,...
127,2022-02-21,3249940,3659618467,1115.0,1135.0,1110.0,1130.0,3476,2454
128,2022-02-22,4981430,5496450079,1115.0,1120.0,1090.0,1110.0,8351,2454
129,2022-02-23,3590396,3986660825,1110.0,1120.0,1100.0,1120.0,4159,2454
130,2022-02-24,8155250,8782736155,1095.0,1105.0,1055.0,1060.0,16505,2454
